In [1]:
from collections import defaultdict, deque
import heapq

In [2]:
import pandas as pd
from collections import defaultdict


df = pd.read_csv('tubedata.csv', header=None)

station_dict = defaultdict(list)
zone_dict = defaultdict(set)
line_dict = defaultdict(set)  


for index, row in df.iterrows():
    start_station = row[0]
    end_station = row[1]
    act_cost = int(row[3])
    zone1 = row[4]
    zone2 = row[5]
    line = row[2]

    
    station_dict[start_station].append((end_station, act_cost, line))
    station_dict[end_station].append((start_station, act_cost, line))

    
    zone_dict[start_station].add(zone1)
    if zone2 != "0":
        zone_dict[start_station].add(zone2)
        zone_dict[end_station].add(zone2)
    else:
        zone_dict[end_station].add(zone1)

    
    line_dict[start_station].add(line)
    line_dict[end_station].add(line)


Implement DFS, BFS and UCS

In [3]:
def bfs(first, last):
    explored = set()
    queue_bfs = deque([([first], 0)]) 
    counter_visited=0

    while queue_bfs: # we will continue to search until there is no nodes to explore
        path, total_time = queue_bfs.popleft()
        node = path[-1]
    
        if node == last:
            return path, total_time,counter_visited

        if node not in explored:
            explored.add(node)
            neighbours = station_dict[node]

            for neighbour, time,line in neighbours:
                if neighbour not in path:  
                    new_path = list(path)
                    new_path.append(neighbour)
                    new_time = total_time + time
                    counter_visited=counter_visited+1
                    queue_bfs.append((new_path, new_time))

    return "No path found.", 0


first_station = "Baker Street"
end_station = "Wembley Park"
route, time_taken,nodes_visited = bfs(first_station, end_station)
print("Route:", route)
print("Total Average Time Taken:", time_taken) 
print("nodes_explored",nodes_visited) 

Route: ['Baker Street', 'Finchley Road', 'Wembley Park']
Total Average Time Taken: 13
nodes_explored 47


In [4]:
def dfs(first, last):
    explored = set()
    stack = [([first], 0)]  
    counter=0
    while stack:
        path, total_time = stack.pop()
        node = path[-1]

        if node == last:
            return path, total_time,counter

        if node not in explored:
            explored.add(node)
            neighbours = station_dict[node]

            for neighbour, time,line in neighbours:
                if neighbour not in path:  # Avoid loops
                    new_path = list(path)
                    new_path.append(neighbour)
                    new_time = total_time + time
                    counter=counter+1
                    stack.append((new_path, new_time))

    return "No path found.", 0


first_station = "Baker Street"
end_station = "Wembley Park"
route, time_taken,nodes_visited = dfs(first_station, end_station)
print("Route:", route)
print("Total Average Time Taken:", time_taken)
print("nodes_explored",nodes_visited) 

Route: ['Baker Street', 'Finchley Road', 'Wembley Park']
Total Average Time Taken: 13
nodes_explored 13


In [5]:
def ucs(first,last):
    visited = set()
    ucs_priority_queue = []
    heapq.heappush(ucs_priority_queue, (0, [first]))  
    counter=0

    while ucs_priority_queue:
        cost, path = heapq.heappop(ucs_priority_queue)
        node = path[-1]

        if node ==last:
            return path, cost,counter

        if node not in visited:
            visited.add(node)

            for neighbour, edge_cost,line in station_dict[node]:
                if neighbour not in visited:
                    new_path = list(path)
                    new_path.append(neighbour)
                    new_cost = cost + edge_cost
                    counter=counter+1
                    heapq.heappush(ucs_priority_queue, (new_cost, new_path))

    return "No path found.", 0


first_station = "Baker Street"
end_station = "Wembley Park"
route, time_taken,nodes_visited = ucs(first_station, end_station)
print("Route:", route)
print("Total Average Time Taken:", time_taken)
print("nodes_explored",nodes_visited) 

Route: ['Baker Street', 'Finchley Road', 'Wembley Park']
Total Average Time Taken: 13
nodes_explored 165


In [6]:
def ucs_with_line_change(first, last, line_change_cost):
    visited = set()
    priority_queue = []
    heapq.heappush(priority_queue, (0, [first], "")) 

    while priority_queue:
        cost, path, current_line = heapq.heappop(priority_queue)
        node = path[-1]

        if node == last:
            return path, cost

        if node not in visited:
            visited.add(node)

            for neighbour, edge_cost, line in station_dict[node]:
                if neighbour not in visited:
                    new_path = list(path)
                    new_path.append(neighbour)
                    new_cost = cost + edge_cost

                    
                    if current_line and line != current_line:
                        new_cost += line_change_cost

                    heapq.heappush(priority_queue, (new_cost, new_path, line))

    return "No path found.", 0


first_station = "Canada Water"
end_station = "Stratford"
route, time_taken = ucs_with_line_change(first_station, end_station,2)
print("Route:", route)
print("Total Average Time Taken:", time_taken)

Route: ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
Total Average Time Taken: 15


In [7]:
# print(station_dict["Victoria"])

 Heuristic search

In [13]:
def zone_cost(current_station, last_station, zone_dict):
    # Implement your zone-based heuristic logic here.
    # For example, higher cost for more zone differences.
    return abs(int(min(list(zone_dict[current_station])))) - (int(min(list(zone_dict[last_station])))) * 1

def best_first_search(first, last, station_dict, line_dict, zone_dict):
    visited = set()
    priority_queue = [(zone_cost(first, last, zone_dict), 0, first, [first])]

    while priority_queue:
        _, current_cost, current_station, path = heapq.heappop(priority_queue)

        if current_station == last:
            return path, current_cost

        if current_station not in visited:
            visited.add(current_station)
            for neighbour, edge_cost, _ in station_dict[current_station]:
                if neighbour not in visited:
                    new_path = path + [neighbour]
                    new_cost = current_cost + edge_cost
                    h_value = zone_cost(neighbour, last, zone_dict)
                    heapq.heappush(priority_queue, (h_value, new_cost, neighbour, new_path))

    return [], 0

# Example usage
first_station = "Canada Water"
end_station = "Stratford"

# Assume zone_dict is defined, mapping stations to their zones
found_path, total_time_taken = best_first_search(first_station, end_station, station_dict, line_dict, zone_dict)
print("Route Found:", found_path)
print("Heuristic cost (including zone cost):", total_time_taken)


Route Found: ['Canada Water', 'Bermondsey', 'London Bridge', 'Bank/Monument', 'Liverpool Street', 'Bethnal Green', 'Mile End', 'Stratford']
Heuristic cost (including zone cost): 18


Genetic  Algorithm

In [9]:
import math
import hashlib
import string

def get_password(student_username, l=10):
  
    options = string.digits + string.ascii_uppercase  + "_"

    h = hashlib.sha256(("ECS759P-AI"+student_username).encode("utf-8"))
    d = h.digest()
    s = ""
    for n in d:
      s += options[n%len(options)]

    return s[0:l]


student_password = get_password('ec23728')
print(student_password)


def distance_function(string_one, string_two):
    score = 0
    for i, j in zip(string_one, string_two):
        
        score += math.sqrt(abs(ord(i) - ord(j)))
    return score


MAX_VALUE = distance_function('0000000000', '__________')

def get_normalised_fitness(list_of_phrases, student_password):
    ordered_dict = dict()
    phrase_to_find = student_password
    for phrase in list_of_phrases:
        ordered_dict[phrase] = 1 - distance_function(phrase, phrase_to_find) / MAX_VALUE
    return ordered_dict


get_normalised_fitness(['2Q4HHHHOTJ', '2HHZQYUOTJ'], student_password)

8YLA8CUGJ0


{'2Q4HHHHOTJ': 0.5076474358311912, '2HHZQYUOTJ': 0.49891385390503706}

In [10]:
import random
import string

def init_population(population_size, password_length):
    characters = string.ascii_uppercase + string.digits + "_"
    population = []
    for _ in range(population_size):
        individual = ''.join(random.choice(characters) for _ in range(password_length))
        population.append(individual)
    return population



In [14]:
def init_population(pop_size, pass_length):
    characters = string.ascii_uppercase + string.digits + "_"
    population = [''.join(random.choice(characters) for _ in range(pass_length)) for _ in range(pop_size)]
    return population

def tournament_selc(population, fitness_scores, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(population, tournament_size)
        winner = max(tournament, key=lambda x: fitness_scores[x])
        selected.append(winner)
    return selected

def crossover(p1, p2):
    crossover_point = random.randint(1, len(p1) - 1)
    offspring1 = p1[:crossover_point] + p2[crossover_point:]
    offspring2 = p2[:crossover_point] + p1[crossover_point:]
    return offspring1, offspring2

def mutate(individual, mutation_rate=0.01):
    characters = string.ascii_uppercase + string.digits + "_"
    mutated_individual = ''.join(random.choice(characters) if random.random() < mutation_rate else char for char in individual)
    return mutated_individual

def update_population(selected, fitness_scores):
    new_population = []
    while len(new_population) < len(selected):
        p1, p2 = random.choices(selected, k=2)
        offspring1, offspring2 = crossover(p1, p2)
        new_population.extend([mutate(offspring1), mutate(offspring2)])
    return new_population[:len(selected)]

def genetic_algorithm(student_password, pop_size=100, pass_length=10, max_generations=1000):
    population = init_population(pop_size, pass_length)

    for generation in range(max_generations):
        fitness_scores = get_normalised_fitness(population, student_password)

        if max(fitness_scores.values()) == 1:
            return [candidate for candidate, fitness in fitness_scores.items() if fitness == 1], generation

        selected = tournament_selc(population, fitness_scores)
        population = update_population(selected, fitness_scores)

    return None, max_generations

found_passwords, generation = genetic_algorithm(student_password)
print(f"Found Password(s): {found_passwords} in Generation: {generation}")


Found Password(s): ['8YLA8CUGJ0'] in Generation: 51


In [15]:
import numpy as np


iterations_needed = [genetic_algorithm(student_password)[1] for _ in range(10)]
print(iterations_needed)
average_iterations = np.mean(iterations_needed)
std_dev_iterations = np.std(iterations_needed)

print(f"Average Number of Iterations: {average_iterations}")
print(f"Standard Deviation: {std_dev_iterations}")


[198, 140, 164, 136, 153, 142, 70, 80, 63, 73]
Average Number of Iterations: 121.9
Standard Deviation: 44.46448020611508
